In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text

In [6]:
data = [
    [8000, 'default'],
    [2000, 'default'],
    [0, 'default'],
    [5000, 'ok'],
    [5000, 'ok'],
    [4000, 'ok'],
    [9000, 'ok'],
    [3000, 'default']
]

df_example = pd.DataFrame(data, columns=['assets', 'status'])

In [8]:
df_example.sort_values('assets')

,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok


In [11]:
# thresholds
Ts = [0, 2000, 3000, 4000, 5000, 8000]
for T in Ts:
    print(T)
    df_left = df_example[df_example.assets <= T]
    df_right = df_example[df_example.assets > T]

    # show misclassification rate -> impurity
    display(df_left)
    print(df_left.status.value_counts(normalize=True))
    display(df_right)
    print(df_right.status.value_counts(normalize=True))
    print()

0


,assets,status
2,0,default


status
default    1.0
Name: proportion, dtype: float64


,assets,status
0,8000,default
1,2000,default
3,5000,ok
4,5000,ok
5,4000,ok
6,9000,ok
7,3000,default


status
ok         0.571429
default    0.428571
Name: proportion, dtype: float64

2000


...

status
default    0.5
ok         0.5
Name: proportion, dtype: float64

8000


,assets,status
0,8000,default
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
7,3000,default


status
default    0.571429
ok         0.428571
Name: proportion, dtype: float64


,assets,status
6,9000,ok


status
ok    1.0
Name: proportion, dtype: float64



|Threshold|Decision Left|Impurity Left|Decision Right|Impurity Right|AVG Impurity|
|---------|--------|---------|---------|----------|---------|
|0|Default|0%|OK|43%| 21.5% |
|2000|Default|0%|OK|33%| 16% |
|3000|Default|0%|OK|20%|10% |
|4000|Default|25%|OK|25%|25% |
|5000|Default|50%|OK|50%|50%|
|8000|Default|43%|OK|0%|21.5%|



The split on 3000 is the best because the average impurity is only 10%. Let's add one more feature `debt`

In [12]:
data = [
    [8000,3000, 'default'],
    [2000, 1000,'default'],
    [0, 1000,'default'],
    [5000,1000, 'ok'],
    [5000, 1000,'ok'],
    [4000,1000,  'ok'],
    [9000, 500,'ok'],
    [3000, 2000, 'default']
]

df_example = pd.DataFrame(data, columns=['assets', 'debt', 'status'])

In [13]:
df_example.sort_values('debt')

,assets,debt,status
6,9000,500,ok
1,2000,1000,default
2,0,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
7,3000,2000,default
0,8000,3000,default


In [17]:
# thresholds for debt = 500, 100, 2000

thresholds = {
    'assets':[0, 2000, 3000, 4000, 5000, 8000],
    'debt': [500, 1000, 2000]
}

for feature in thresholds:
    print(feature)
    for T in thresholds[feature]:
        print(T)
        df_left = df_example[df_example[feature] <= T]
        df_right = df_example[df_example[feature] > T]

        # show misclassification rate -> impurity
        display(df_left)
        print(df_left.status.value_counts(normalize=True))
        display(df_right)
        print(df_right.status.value_counts(normalize=True))
        print()
    print('===================')
    print()
    

assets
0


,assets,debt,status
2,0,1000,default


status
default    1.0
Name: proportion, dtype: float64


,assets,debt,status
0,8000,3000,default
1,2000,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
6,9000,500,ok
7,3000,2000,default


status
ok         0.571429
default    0.428571
Name: proportion, dtype: float64

2000


...

status
default    1.0
Name: proportion, dtype: float64

2000


,assets,debt,status
1,2000,1000,default
2,0,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
6,9000,500,ok
7,3000,2000,default


status
ok         0.571429
default    0.428571
Name: proportion, dtype: float64


,assets,debt,status
0,8000,3000,default


status
default    1.0
Name: proportion, dtype: float64




__assets__
|Threshold|Decision Left|Impurity Left|Decision Right|Impurity Right|AVG Impurity|
|---------|--------|---------|---------|----------|---------|
|0|Default|0%|OK|43%| 21.5% |
|2000|Default|0%|OK|33%| 16% |
|3000|Default|0%|OK|20%|10% |
|4000|Default|25%|OK|25%|25% |
|5000|Default|50%|OK|50%|50%|
|8000|Default|43%|OK|0%|21.5%|


__debt__
|Threshold|Decision Left|Impurity Left|Decision Right|Impurity Right|AVG Impurity|
|---------|--------|---------|---------|----------|---------|
|500|OK|0%|Default|43%| 21% |
|1000|OK|33%|Default|0%| 16.5% |
|2000|OK|43%|Default|0%|25% |




Assets -> 3000 still shows the lowest average impurity. So it's a first split

__Stop criterias__

- Group is already pure
- Tree reached a depth limit
- Min size of the group: group is too small

#### DT algorithm

1. Find the best split
2. Stop if `max_depth` is reached
3. If left is sufficiently lagrge AND not pure:
    - repeat for the left
4. If right is sufficiently lagrge AND not pure:
    - repeat for the right